# DaisyUI Reset Fixup

MonsterUI composes DaisyUI, FrankenUI, and Tailwind. DaisyUI 5 introduced new base-layer resets that trump Franken's tokens and component defaults. This notebook catalogs those collisions, synthesizes a scoped overlay (with an opt-in sentinel and universal guard), and exports helpers via `monsterui.fixup` so `core` can inject the fixup stylesheet at runtime.


## Export to `monsterui/fixup.py`

In [ ]:
#|default_exp fixup

## Imports

In [48]:
from __future__ import annotations

In [49]:
#| export
from dataclasses import dataclass
from typing import Iterable, Iterator, Sequence

from collections import defaultdict
from functools import lru_cache
from pathlib import Path
import re
from textwrap import indent

import httpx
import tinycss2


In [ ]:
#| export
try:
    MODULE_DIR = Path(__file__).resolve().parent
except NameError:  # pragma: no cover - notebook execution environment
    cwd = Path.cwd()
    MODULE_DIR = cwd / 'monsterui' if (cwd / 'monsterui').exists() else cwd
ASSET_ROOT = MODULE_DIR.parent


def _read_local_asset(name: str) -> str | None:
    "Read a local CSS snapshot bundled with the repo/package when available."
    path = ASSET_ROOT / name
    if path.exists():
        return path.read_text()
    return None


In [50]:
from fastcore.all import *
from monsterui.foundations import *

In [51]:
from nbdev.showdoc import *

## App

In [52]:
#| export
DAISY_CDN_URL = "https://cdn.jsdelivr.net/npm/daisyui@5"
FRANKEN_CSS_URL = "https://cdn.jsdelivr.net/npm/franken-ui@2.1.1/dist/css/core.min.css"
RESET_PATTERNS = {
    "universal": re.compile(r"(?<![\w-])\*(?![\w-])"),
    "where-universal": re.compile(r":where\(\s*\*\s*\)"),
    "pseudo-elements": re.compile(r"\*\s*::(before|after)"),
    "file-selector": re.compile(r"::file-selector-button"),
    "form-controls": re.compile(r"([,\s>+~(]|^)(button|input|textarea|select|optgroup)\b"),
    "spinner": re.compile(r"\buk-spinner\b"),
}


In [53]:
#| export
UNIVERSAL_RESET_SELECTOR = ":where(*, *::before, *::after, ::backdrop, ::file-selector-button)"
UNIVERSAL_GUARD_PROPERTIES = (
    'border',
    'border-width',
    'border-style',
    'border-color',
    'margin',
    'padding',
    'box-sizing',
)
DEFAULT_ACTIVATION_SENTINEL = ':is([data-monsterui-franken], .uk-theme-slate, .uk-theme-stone, .uk-theme-gray, .uk-theme-neutral, .uk-theme-red, .uk-theme-rose, .uk-theme-orange, .uk-theme-green, .uk-theme-blue, .uk-theme-yellow, .uk-theme-violet, .uk-theme-zinc)'
DEFAULT_LAYER_NAME = None


In [54]:
#| export
@dataclass(frozen=True)
class Declaration:
    name: str
    value: str
    important: bool = False

    def as_css(self) -> str:
        suffix = " !important" if self.important else ""
        return f"{self.name}: {self.value}{suffix};"


@dataclass
class RuleRecord:
    selector: str
    declarations: tuple[Declaration, ...]
    contexts: tuple[str, ...]
    categories: tuple[str, ...]

    @property
    def properties(self) -> tuple[str, ...]:
        "Return the ordered property names declared in this rule."
        if not self.declarations:
            return tuple()
        return tuple(decl.name for decl in self.declarations)


In [55]:
#| export
def fetch_css(url: str) -> str:
    'Download a CSS asset from jsDelivr and ensure we got CSS.'
    resp = httpx.get(url, follow_redirects=True, timeout=30)
    status = resp.status_code
    if status != 200:
        raise ValueError(f"Unexpected HTTP status {status} for {url}")
    content_type = resp.headers.get('content-type', '').lower()
    if 'text/css' not in content_type:
        raise ValueError(f"Unexpected content type {content_type!r} for {url}")
    return resp.text


In [56]:
#| export
def _serialize(tokens) -> str:
    if not tokens:
        return ''
    return tinycss2.serialize(tokens).strip()


In [57]:
#| export
def _parse_declarations(content) -> tuple[Declaration, ...]:
    if not content:
        return tuple()
    decls = tinycss2.parse_declaration_list(content, skip_comments=True, skip_whitespace=True)
    parsed: list[Declaration] = []
    for decl in decls:
        if not isinstance(decl, tinycss2.ast.Declaration):
            continue
        name = decl.name.strip()
        if not name:
            continue
        value = _serialize(decl.value)
        if not value:
            continue
        parsed.append(Declaration(name=name, value=value, important=bool(decl.important)))
    return tuple(parsed)


In [58]:

#| export
def _walk_rules(nodes, contexts: Sequence[str] | None = None):
    contexts = tuple(contexts or ())
    for node in nodes:
        if isinstance(node, tinycss2.ast.QualifiedRule):
            selector = _serialize(node.prelude)
            declarations = _parse_declarations(node.content)
            if not declarations:
                continue
            yield RuleRecord(selector=selector, declarations=declarations, contexts=contexts, categories=tuple())
        elif isinstance(node, tinycss2.ast.AtRule):
            if node.content is None:
                continue
            at_kw = getattr(node, "at_keyword", "") or ""
            ctx_name = f"@{at_kw.lstrip('@')}" if at_kw else "@unknown"
            prelude = _serialize(node.prelude)
            if prelude:
                ctx_name = f"{ctx_name} {prelude}"
            inner_rules = tinycss2.parse_rule_list(
                node.content, skip_whitespace=True, skip_comments=True
            )
            yield from _walk_rules(inner_rules, contexts=contexts + (ctx_name,))


In [59]:
#| export
def _categorize(selector: str) -> tuple[str, ...]:
    matches = []
    for name, pattern in RESET_PATTERNS.items():
        if pattern.search(selector):
            matches.append(name)
    return tuple(matches)


In [60]:

#| export
def collect_rules(css_text: str, *, categorize: bool = True) -> list[RuleRecord]:
    stylesheet = tinycss2.parse_stylesheet(css_text, skip_comments=True, skip_whitespace=True)
    records: list[RuleRecord] = []
    for rule in _walk_rules(stylesheet):
        categories = _categorize(rule.selector) if categorize else tuple()
        records.append(
            RuleRecord(
                selector=rule.selector,
                declarations=rule.declarations,
                contexts=rule.contexts,
                categories=categories,
            )
        )
    return records


def collect_reset_rules(css_text: str, *, only_categorized: bool = True) -> list[RuleRecord]:
    'Return Daisy rules that match reset patterns when requested.'
    records = collect_rules(css_text)
    if only_categorized:
        return [record for record in records if record.categories]
    return records


In [61]:
#| export
def summarize_property_impact(records: Iterable[RuleRecord]) -> dict[str, list[str]]:
    summary: dict[str, set[str]] = {}
    for rule in records:
        for prop in rule.properties:
            summary.setdefault(prop, set()).add(rule.selector)
    return {prop: sorted(selectors) for prop, selectors in summary.items()}


In [62]:

#| export
def render_text_report(records: Sequence[RuleRecord]) -> str:
    lines: list[str] = [f"Matched {len(records)} reset-like rules:"]
    if not records:
        return ''.join(lines)
    summary = summarize_property_impact(records)
    lines.append('Property impact summary:')
    for prop, selectors in sorted(summary.items()):
        lines.append(f"- {prop} (selectors: {len(selectors)})")
    lines.append('Detailed rules:')
    for rule in records:
        ctx = ' > '.join(rule.contexts) if rule.contexts else '(root)'
        lines.append(f"* Selector: {rule.selector}")
        lines.append(f"  Context: {ctx}")
        lines.append(f"  Categories: {', '.join(rule.categories)}")
        lines.append(f"  Properties ({len(rule.properties)}): {', '.join(rule.properties)}")
        lines.append('')
    return ''.join(lines)


In [63]:
#| export
def daisy_reset_report(url: str = DAISY_CDN_URL, output: str = 'text'):
    css_text = fetch_css(url)
    records = collect_reset_rules(css_text)
    if output == 'json':
        return {
            'rules': [
                {
                    'selector': r.selector,
                    'properties': list(r.properties),
                    'contexts': list(r.contexts),
                    'categories': list(r.categories),
                }
                for r in records
            ],
            'properties': summarize_property_impact(records),
        }
    return render_text_report(records)


In [64]:
report_preview = daisy_reset_report()
print(''.join(report_preview.splitlines()[:40]))


Matched 13 reset-like rules:Property impact summary:- --border (selectors: 2)- --color-accent (selectors: 2)- --color-accent-content (selectors: 2)- --color-base-100 (selectors: 2)- --color-base-200 (selectors: 2)- --color-base-300 (selectors: 2)- --color-base-content (selectors: 2)- --color-error (selectors: 2)- --color-error-content (selectors: 2)- --color-info (selectors: 2)- --color-info-content (selectors: 2)- --color-neutral (selectors: 2)- --color-neutral-content (selectors: 2)- --color-primary (selectors: 2)- --color-primary-content (selectors: 2)- --color-secondary (selectors: 2)- --color-secondary-content (selectors: 2)- --color-success (selectors: 2)- --color-success-content (selectors: 2)- --color-warning (selectors: 2)- --color-warning-content (selectors: 2)- --depth (selectors: 2)- --noise (selectors: 2)- --radius-box (selectors: 2)- --radius-field (selectors: 2)- --radius-selector (selectors: 2)- --size-field (selectors: 2)- --size-selector (selectors: 2)- appearance (se

In [65]:
#| export
def split_selectors(selector: str) -> tuple[str, ...]:
    'Split a selector list into individual selectors.'
    if not selector:
        return tuple()
    parts = [part.strip() for part in selector.split(',')]
    return tuple(part for part in parts if part)


In [66]:
#| export
def _selectors_from_nodes(nodes):
    for node in nodes:
        if isinstance(node, tinycss2.ast.QualifiedRule):
            for part in split_selectors(_serialize(node.prelude)):
                yield part
        elif isinstance(node, tinycss2.ast.AtRule) and node.content:
            inner = tinycss2.parse_rule_list(node.content, skip_whitespace=True, skip_comments=True)
            yield from _selectors_from_nodes(inner)


In [67]:
#| export
def collect_selectors_from_css(css_text: str) -> set[str]:
    'Collect every selector defined within the provided CSS text.'
    rules = tinycss2.parse_stylesheet(css_text, skip_comments=True, skip_whitespace=True)
    return set(_selectors_from_nodes(rules))


### FrankenUI

In [68]:
#| export
def collect_franken_selectors(url: str = FRANKEN_CSS_URL) -> set[str]:
    'Download the FrankenUI stylesheet and return its selector set.'
    css_text = fetch_css(url)
    return collect_selectors_from_css(css_text)


In [69]:
#| export
def find_franken_overlaps(records: Sequence[RuleRecord], franken_selectors: set[str]):
    overlaps = []
    for record in records:
        for selector in split_selectors(record.selector):
            if selector in franken_selectors:
                overlaps.append({
                    'selector': selector,
                    'properties': record.properties,
                    'categories': record.categories,
                    'contexts': record.contexts,
                })
    return overlaps


### Reset overlay toolkit

Build selector profiles, diff Daisy resets, and synthesize Franken overlays.


In [70]:

#| export
@dataclass
class SelectorProfile:
    selector: str
    contexts: tuple[str, ...]
    categories: tuple[str, ...]
    declarations: dict[str, Declaration]


#| export
@dataclass
class ConflictRecord:
    selector: str
    contexts: tuple[str, ...]
    categories: tuple[str, ...]
    properties: tuple[tuple[str, Declaration, Declaration], ...]


In [71]:

#| export
def _iter_selector_records(records: Sequence[RuleRecord], *, require_categories: bool = False) -> Iterator[RuleRecord]:
    for record in records:
        if require_categories and not record.categories:
            continue
        selectors = split_selectors(record.selector) or (record.selector,)
        for selector in selectors:
            yield RuleRecord(
                selector=selector,
                declarations=record.declarations,
                contexts=record.contexts,
                categories=record.categories,
            )


#| export
def build_selector_profiles(records: Sequence[RuleRecord], *, require_categories: bool = False) -> dict[tuple[str, tuple[str, ...]], SelectorProfile]:
    profiles: dict[tuple[str, tuple[str, ...]], dict] = {}
    for record in _iter_selector_records(records, require_categories=require_categories):
        key = (record.selector, record.contexts)
        entry = profiles.setdefault(
            key,
            {
                'selector': record.selector,
                'contexts': record.contexts,
                'categories': [],
                'declarations': {},
            },
        )
        for category in record.categories:
            if category not in entry['categories']:
                entry['categories'].append(category)
        for declaration in record.declarations:
            entry['declarations'][declaration.name] = declaration
    return {
        key: SelectorProfile(
            selector=data['selector'],
            contexts=data['contexts'],
            categories=tuple(data['categories']),
            declarations=data['declarations'],
        )
        for key, data in profiles.items()
    }


In [72]:

#| export
def _choose_profile_match(options: Sequence[SelectorProfile], contexts: tuple[str, ...]) -> SelectorProfile:
    if not options:
        raise ValueError('No selector profiles provided')
    for profile in options:
        if profile.contexts == contexts:
            return profile
    for profile in options:
        if not profile.contexts:
            return profile
    return options[0]


In [73]:

#| export
def _is_reset_profile(profile: SelectorProfile) -> bool:
    if profile.categories:
        return True
    return any(ctx.startswith('@layer base') for ctx in profile.contexts)


In [74]:

#| export
def find_reset_conflicts(daisy_css: str, franken_css: str) -> list[ConflictRecord]:
    daisy_records = collect_reset_rules(daisy_css, only_categorized=False)
    franken_records = collect_rules(franken_css, categorize=False)
    daisy_profiles = build_selector_profiles(daisy_records, require_categories=False)
    franken_profiles = build_selector_profiles(franken_records, require_categories=False)

    by_selector: dict[str, list[SelectorProfile]] = defaultdict(list)
    for profile in franken_profiles.values():
        by_selector[profile.selector].append(profile)

    conflicts: list[ConflictRecord] = []
    for profile in daisy_profiles.values():
        if not _is_reset_profile(profile):
            continue
        matches = by_selector.get(profile.selector)
        if not matches:
            continue
        candidate = _choose_profile_match(matches, profile.contexts)
        overlapping = []
        for name, daisy_decl in profile.declarations.items():
            target_decl = candidate.declarations.get(name)
            if target_decl is None:
                continue
            if daisy_decl.value == target_decl.value and daisy_decl.important == target_decl.important:
                continue
            overlapping.append((name, daisy_decl, target_decl))
        if overlapping:
            conflicts.append(
                ConflictRecord(
                    selector=profile.selector,
                    contexts=profile.contexts,
                    categories=profile.categories,
                    properties=tuple(overlapping),
                )
            )
    return conflicts


In [75]:


#| export
def _wrap_contexts(block: str, contexts: tuple[str, ...], indent_str: str = '  ') -> str:
    wrapped = block
    for context in reversed(contexts):
        inner = indent(wrapped.rstrip(), indent_str)
        wrapped = f"{context} {{{inner}}}"
    return wrapped


#| export
def _scoped_selector(selector: str, scope: str | None) -> str:
    if not scope:
        return selector
    scope = scope.strip()
    if not scope:
        return selector
    return f"{scope} {selector}".strip()


#| export
def _apply_activation(selector: str, activation: str | None) -> str:
    if not activation:
        return selector
    guard = activation.strip()
    if not guard:
        return selector
    base = selector.strip()
    if not base:
        return selector
    if base in {'html', 'body'}:
        return f"{base}{guard}"
    if base.startswith(':host'):
        if base == ':host':
            return f":host({guard})"
        if base.startswith(':host('):
            return f":is(:host({guard}), {base})"
    return f"{guard} {base}".strip()


#| export
def generate_universal_guard(
    *,
    activation_selector: str | None,
    scope: str | None = None,
    properties: Sequence[str] | None = None,
    indent_str: str = '  ',
) -> str:
    if not activation_selector and not scope:
        raise ValueError('Universal guard requires an activation selector or explicit scope')
    props = tuple(properties or UNIVERSAL_GUARD_PROPERTIES)
    base_selector = _apply_activation(UNIVERSAL_RESET_SELECTOR, activation_selector)
    selector = _scoped_selector(base_selector, scope)
    lines = ''.join(f"{indent_str}{prop}: revert-layer;" for prop in props)
    block = f"{selector} {{{lines}}}"
    return _wrap_contexts(block, ('@layer base',), indent_str=indent_str)


#| export
def generate_overlay_rules(
    conflicts: Sequence[ConflictRecord],
    *,
    scope: str | None = None,
    activation_selector: str | None = None,
    indent_str: str = '  ',
) -> list[str]:
    rules: list[str] = []
    for conflict in conflicts:
        declarations = [target for _, _, target in conflict.properties]
        if not declarations:
            continue
        lines = ''.join(f"{indent_str}{decl.as_css()}" for decl in declarations)
        base_selector = _apply_activation(conflict.selector, activation_selector)
        selector = _scoped_selector(base_selector, scope)
        block = f"{selector} {{{lines}}}"
        rules.append(_wrap_contexts(block, conflict.contexts, indent_str=indent_str))
    return rules


#| export
def build_franken_overlay(
    daisy_css: str,
    franken_css: str,
    *,
    scope: str | None = None,
    activation_selector: str | None = None,
    layer_name: str | None = 'monsterui.franken-overrides',
    indent_str: str = '  ',
    universal_guard: bool = False,
    guard_properties: Sequence[str] | None = None,
) -> dict:
    conflicts = find_reset_conflicts(daisy_css, franken_css)
    rules = generate_overlay_rules(
        conflicts,
        scope=scope,
        activation_selector=activation_selector,
        indent_str=indent_str,
    )
    extra_blocks: list[str] = []
    if universal_guard:
        extra_blocks.append(
            generate_universal_guard(
                activation_selector=activation_selector,
                scope=scope,
                properties=guard_properties,
                indent_str=indent_str,
            )
        )
    css_chunks = [rule.strip() for rule in rules if rule.strip()]
    css_chunks.extend(block.strip() for block in extra_blocks if block and block.strip())
    css_body = '\n\n'.join(css_chunks)
    if layer_name and css_body:
        css_text = f"@layer {layer_name} {{{indent(css_body, indent_str)}}}"
    else:
        css_text = css_body
    return {
        'css': css_text,
        'rules': conflicts,
        'rule_count': len(rules),
    }


In [ ]:
#| export
@lru_cache(maxsize=None)
def _fetch_stylesheet(url: str) -> str:
    return fetch_css(url)


def _load_stylesheet(source: str | None, *, url: str, fallback: str) -> str:
    if source:
        return source
    last_error: Exception | None = None
    if url:
        try:
            return _fetch_stylesheet(url)
        except Exception as exc:  # pragma: no cover - network variability
            last_error = exc
    local = _read_local_asset(fallback)
    if local is not None:
        return local
    if last_error is not None:
        raise last_error
    raise FileNotFoundError(f"Unable to load stylesheet from {url!r} and no fallback {fallback!r}")


### DaisyUI

In [76]:
daisy_css = fetch_css(DAISY_CDN_URL)
daisy_records = collect_reset_rules(daisy_css)
print(f"Matched {len(daisy_records)} Daisy reset-like rules")
franken_selectors = collect_franken_selectors()
print(f"Collected {len(franken_selectors)} Franken selectors")
overlaps = find_franken_overlaps(daisy_records, franken_selectors)
print(f"Found {len(overlaps)} overlapping selectors")
for sample in overlaps[:10]:
    print('-', sample['selector'], sample['properties'])


Matched 13 Daisy reset-like rules
Collected 1031 Franken selectors
Found 19 overlapping selectors
- :root ('color-scheme', '--color-base-100', '--color-base-200', '--color-base-300', '--color-base-content', '--color-primary', '--color-primary-content', '--color-secondary', '--color-secondary-content', '--color-accent', '--color-accent-content', '--color-neutral', '--color-neutral-content', '--color-info', '--color-info-content', '--color-success', '--color-success-content', '--color-warning', '--color-warning-content', '--color-error', '--color-error-content', '--radius-selector', '--radius-field', '--radius-box', '--size-selector', '--size-field', '--border', '--depth', '--noise')
- * ('box-sizing', 'border', 'margin', 'padding')
- ::backdrop ('box-sizing', 'border', 'margin', 'padding')
- ::file-selector-button ('box-sizing', 'border', 'margin', 'padding')
- ::file-selector-button ('font', 'font-feature-settings', 'font-variation-settings', 'letter-spacing', 'color', 'background')
- 

#### Franken reset overlay

Compute conflict metadata and emit an override stylesheet.


In [77]:

franken_css = fetch_css(FRANKEN_CSS_URL)
conflicts = find_reset_conflicts(daisy_css, franken_css)
print(f"Detected {len(conflicts)} conflicting selectors")
prop_count = sum(len(record.properties) for record in conflicts)
print(f"Captured {prop_count} property overrides")


Detected 4 conflicting selectors
Captured 6 property overrides


In [78]:

for record in conflicts:
    ctx = ' > '.join(record.contexts) if record.contexts else '(root)'
    cats = ', '.join(record.categories) if record.categories else 'uncategorized'
    print(f"{record.selector} | Context: {ctx} | Categories: {cats}")
    for name, daisy_decl, franken_decl in record.properties:
        print(f"  - {name}")
        print(f"      Daisy  : {daisy_decl.as_css()}")
        print(f"      Franken: {franken_decl.as_css()}")
    print()


:host | Context: @layer base | Categories: uncategorized
  - font-family
      Daisy  : font-family: var(--default-font-family,ui-sans-serif,system-ui,sans-serif,"Apple Color Emoji","Segoe UI Emoji","Segoe UI Symbol","Noto Color Emoji");
      Franken: font-family: ui-sans-serif,system-ui,sans-serif,"Apple Color Emoji","Segoe UI Emoji","Segoe UI Symbol","Noto Color Emoji";
  - -webkit-tap-highlight-color
      Daisy  : -webkit-tap-highlight-color: #0000;
      Franken: -webkit-tap-highlight-color: transparent;

html | Context: @layer base | Categories: uncategorized
  - font-family
      Daisy  : font-family: var(--default-font-family,ui-sans-serif,system-ui,sans-serif,"Apple Color Emoji","Segoe UI Emoji","Segoe UI Symbol","Noto Color Emoji");
      Franken: font-family: ui-sans-serif,system-ui,sans-serif,"Apple Color Emoji","Segoe UI Emoji","Segoe UI Symbol","Noto Color Emoji";
  - -webkit-tap-highlight-color
      Daisy  : -webkit-tap-highlight-color: #0000;
      Franken: -webkit-ta

In [79]:

overlay = build_franken_overlay(daisy_css, franken_css)
overlay_style = Style(overlay['css'])
print(f"Overlay emits {overlay['rule_count']} rewrite blocks")
print(overlay_style)


Overlay emits 4 rewrite blocks
<style>@layer monsterui.franken-overrides {  @layer base {  :host {  font-family: ui-sans-serif,system-ui,sans-serif,"Apple Color Emoji","Segoe UI Emoji","Segoe UI Symbol","Noto Color Emoji";  -webkit-tap-highlight-color: transparent;}}@layer base {  html {  font-family: ui-sans-serif,system-ui,sans-serif,"Apple Color Emoji","Segoe UI Emoji","Segoe UI Symbol","Noto Color Emoji";  -webkit-tap-highlight-color: transparent;}}@layer base {  body {  line-height: var(--uk-global-leading);}}@layer base {  ::placeholder {  color: color-mix(in oklab,currentColor 50%,transparent);}}}</style>


In [ ]:
#| export
def build_franken_fixup_css(
    *,
    activation_selector: str = DEFAULT_ACTIVATION_SENTINEL,
    scope: str | None = None,
    layer_name: str | None = DEFAULT_LAYER_NAME,
    include_universal_guard: bool = True,
    guard_properties: Sequence[str] | None = None,
    daisy_css: str | None = None,
    franken_css: str | None = None,
    daisy_url: str = DAISY_CDN_URL,
    franken_url: str = FRANKEN_CSS_URL,
    daisy_fallback: str = 'daisyui5.full.min.prettier.css',
    franken_fallback: str = 'frankenui.core.min.prettier.css',
) -> str:
    "Build the opt-in Franken overlay that reverts Daisy resets inside a sentinel scope."
    guard_props = tuple(guard_properties or UNIVERSAL_GUARD_PROPERTIES)
    daisy_sheet = _load_stylesheet(daisy_css, url=daisy_url, fallback=daisy_fallback)
    franken_sheet = _load_stylesheet(franken_css, url=franken_url, fallback=franken_fallback)
    overlay = build_franken_overlay(
        daisy_sheet,
        franken_sheet,
        scope=scope,
        activation_selector=activation_selector,
        layer_name=layer_name,
        universal_guard=include_universal_guard,
        guard_properties=guard_props,
    )
    return overlay['css']


### Scoped overlay activation

Attach a sentinel (for example `data-monsterui-franken` or `.uk-theme-blue`) to the Franken root. The overlay builder now applies both the targeted property rewrites *and* a universal guard that reverts Daisy's blanket reset only inside that sentinel, so Daisy keeps control elsewhere.


In [80]:
scoped_overlay_css = build_franken_fixup_css()
print("Scoped overlay CSS length:", len(scoped_overlay_css))
print(Style(scoped_overlay_css))


Scoped overlay emits 4 targeted rewrite blocks
<style>@layer monsterui.franken-overrides.scoped {  @layer base {  :host([data-monsterui-franken]) {  font-family: ui-sans-serif,system-ui,sans-serif,"Apple Color Emoji","Segoe UI Emoji","Segoe UI Symbol","Noto Color Emoji";  -webkit-tap-highlight-color: transparent;}}@layer base {  html[data-monsterui-franken] {  font-family: ui-sans-serif,system-ui,sans-serif,"Apple Color Emoji","Segoe UI Emoji","Segoe UI Symbol","Noto Color Emoji";  -webkit-tap-highlight-color: transparent;}}@layer base {  body[data-monsterui-franken] {  line-height: var(--uk-global-leading);}}@layer base {  [data-monsterui-franken] ::placeholder {  color: color-mix(in oklab,currentColor 50%,transparent);}}@layer base {  [data-monsterui-franken] :where(*, *::before, *::after, ::backdrop, ::file-selector-button) {  border: revert-layer;  border-width: revert-layer;  border-style: revert-layer;  border-color: revert-layer;  margin: revert-layer;  padding: revert-layer;  b